In [46]:
import astropy as ast
from astropy.coordinates import solar_system_ephemeris, EarthLocation, GeocentricTrueEcliptic, get_body, SkyCoord, Distance
from astroplan.moon import moon_phase_angle
from collections import defaultdict
import pandas as pd
import numpy as np

### Generating Data From Celestial Bodies

In [3]:
BODY_NAMES = ['mercury', 'venus', 'mars', 'jupiter', 'saturn', 'moon', 'sun']

In [34]:
def get_coordinates(body):
    # Takes a Skycoord object, returns (theta, phi, r, x, y, z) in (deg, deg, AU, AU, AU, AU)
    angles = [float(i) for i in body.to_string().split(' ')]
    body_dist_string = body.distance.to_string()
    r = float(body_dist_string[:-3])
    units = body_dist_string[-2:]
    if units == 'km':
        r /= 1.496e+8
    
    phi = angles[0]
    theta = angles[1]

    c = body.cartesian
    x = c.x.value
    y = c.y.value
    z = c.z.value

    body_dist_string = body.distance.to_string()
    units = body_dist_string[-2:]
    if units == 'km':
        x /= 1.496e+8
        y /= 1.496e+8
        z /= 1.496e+8
    return (phi, theta, r, x, y, z)

In [35]:
def add_noise(coords):
    # Takes a tuple of (theta, phi, r, x, y, z) in (deg, deg, AU, AU, AU, AU)
    # and returns a noisified version of the data
    # Currently doesn't add noise to r
    theta, phi, r, x, y, z = coords
    return (theta + np.random.normal(0,1), phi + np.random.normal(0,1), r + np.random.normal(0, 0.1),
            x + np.random.normal(0, 0.1), y + np.random.normal(0, 0.1), z + np.random.normal(0, 0.1))

times = pd.date_range(start="1850-01-01-00-00-00", end="2000-01-01-00-00-00", freq='5D')
loc = EarthLocation.of_site('medicina')
rows = defaultdict(list)


In [15]:
for time in times:
    print(time)
    time = ast.time.Time(time.to_pydatetime())
    bodies = []
    
    with solar_system_ephemeris.set('builtin'):
        for body_name in BODY_NAMES:
            bodies.append(get_body(body_name, time, loc))

    rows['time'].append(time)
    rows['location'].append(str(loc))
    rows['moon_phase'].append(moon_phase_angle(time).value)

    for body_name, body in zip(BODY_NAMES, bodies):
        coordinates = add_noise(get_coordinates(body))
        coord_strings = ['theta', 'phi', 'r', 'x', 'y', 'z']
        for i in range(len(coord_strings)):
            c = coordinates[i]
            rows[body_name + '_' + coord_strings[i]].append(c)

celestial_bodies = pd.DataFrame(rows)
celestial_bodies.to_csv('data.csv', index=False)

1850-01-01 00:00:00+00:00


1850-01-06 00:00:00+00:00
1850-01-11 00:00:00+00:00
1850-01-16 00:00:00+00:00
1850-01-21 00:00:00+00:00
1850-01-26 00:00:00+00:00
1850-01-31 00:00:00+00:00
1850-02-05 00:00:00+00:00
1850-02-10 00:00:00+00:00
1850-02-15 00:00:00+00:00
1850-02-20 00:00:00+00:00
1850-02-25 00:00:00+00:00
1850-03-02 00:00:00+00:00
1850-03-07 00:00:00+00:00
1850-03-12 00:00:00+00:00
1850-03-17 00:00:00+00:00
1850-03-22 00:00:00+00:00
1850-03-27 00:00:00+00:00
1850-04-01 00:00:00+00:00
1850-04-06 00:00:00+00:00
1850-04-11 00:00:00+00:00
1850-04-16 00:00:00+00:00
1850-04-21 00:00:00+00:00
1850-04-26 00:00:00+00:00
1850-05-01 00:00:00+00:00
1850-05-06 00:00:00+00:00
1850-05-11 00:00:00+00:00
1850-05-16 00:00:00+00:00
1850-05-21 00:00:00+00:00
1850-05-26 00:00:00+00:00
1850-05-31 00:00:00+00:00
1850-06-05 00:00:00+00:00
1850-06-10 00:00:00+00:00
1850-06-15 00:00:00+00:00
1850-06-20 00:00:00+00:00
1850-06-25 00:00:00+00:00
1850-06-30 00:00:00+00:00
1850-07-05 00:00:00+00:00
1850-07-10 00:00:00+00:00
1850-07-15 0

1854-05-05 00:00:00+00:00
1854-05-10 00:00:00+00:00
1854-05-15 00:00:00+00:00
1854-05-20 00:00:00+00:00
1854-05-25 00:00:00+00:00
1854-05-30 00:00:00+00:00
1854-06-04 00:00:00+00:00
1854-06-09 00:00:00+00:00
1854-06-14 00:00:00+00:00
1854-06-19 00:00:00+00:00
1854-06-24 00:00:00+00:00
1854-06-29 00:00:00+00:00
1854-07-04 00:00:00+00:00
1854-07-09 00:00:00+00:00
1854-07-14 00:00:00+00:00
1854-07-19 00:00:00+00:00
1854-07-24 00:00:00+00:00
1854-07-29 00:00:00+00:00
1854-08-03 00:00:00+00:00
1854-08-08 00:00:00+00:00
1854-08-13 00:00:00+00:00
1854-08-18 00:00:00+00:00
1854-08-23 00:00:00+00:00
1854-08-28 00:00:00+00:00
1854-09-02 00:00:00+00:00
1854-09-07 00:00:00+00:00
1854-09-12 00:00:00+00:00
1854-09-17 00:00:00+00:00
1854-09-22 00:00:00+00:00
1854-09-27 00:00:00+00:00
1854-10-02 00:00:00+00:00
1854-10-07 00:00:00+00:00
1854-10-12 00:00:00+00:00
1854-10-17 00:00:00+00:00
1854-10-22 00:00:00+00:00
1854-10-27 00:00:00+00:00
1854-11-01 00:00:00+00:00
1854-11-06 00:00:00+00:00
1854-11-11 0

1858-09-01 00:00:00+00:00
1858-09-06 00:00:00+00:00
1858-09-11 00:00:00+00:00
1858-09-16 00:00:00+00:00
1858-09-21 00:00:00+00:00
1858-09-26 00:00:00+00:00
1858-10-01 00:00:00+00:00
1858-10-06 00:00:00+00:00
1858-10-11 00:00:00+00:00
1858-10-16 00:00:00+00:00
1858-10-21 00:00:00+00:00
1858-10-26 00:00:00+00:00
1858-10-31 00:00:00+00:00
1858-11-05 00:00:00+00:00
1858-11-10 00:00:00+00:00
1858-11-15 00:00:00+00:00
1858-11-20 00:00:00+00:00
1858-11-25 00:00:00+00:00
1858-11-30 00:00:00+00:00
1858-12-05 00:00:00+00:00
1858-12-10 00:00:00+00:00
1858-12-15 00:00:00+00:00
1858-12-20 00:00:00+00:00
1858-12-25 00:00:00+00:00
1858-12-30 00:00:00+00:00
1859-01-04 00:00:00+00:00
1859-01-09 00:00:00+00:00
1859-01-14 00:00:00+00:00
1859-01-19 00:00:00+00:00
1859-01-24 00:00:00+00:00
1859-01-29 00:00:00+00:00
1859-02-03 00:00:00+00:00
1859-02-08 00:00:00+00:00
1859-02-13 00:00:00+00:00
1859-02-18 00:00:00+00:00
1859-02-23 00:00:00+00:00
1859-02-28 00:00:00+00:00
1859-03-05 00:00:00+00:00
1859-03-10 0

1862-12-29 00:00:00+00:00
1863-01-03 00:00:00+00:00
1863-01-08 00:00:00+00:00
1863-01-13 00:00:00+00:00
1863-01-18 00:00:00+00:00
1863-01-23 00:00:00+00:00
1863-01-28 00:00:00+00:00
1863-02-02 00:00:00+00:00
1863-02-07 00:00:00+00:00
1863-02-12 00:00:00+00:00
1863-02-17 00:00:00+00:00
1863-02-22 00:00:00+00:00
1863-02-27 00:00:00+00:00
1863-03-04 00:00:00+00:00
1863-03-09 00:00:00+00:00
1863-03-14 00:00:00+00:00
1863-03-19 00:00:00+00:00
1863-03-24 00:00:00+00:00
1863-03-29 00:00:00+00:00
1863-04-03 00:00:00+00:00
1863-04-08 00:00:00+00:00
1863-04-13 00:00:00+00:00
1863-04-18 00:00:00+00:00
1863-04-23 00:00:00+00:00
1863-04-28 00:00:00+00:00
1863-05-03 00:00:00+00:00
1863-05-08 00:00:00+00:00
1863-05-13 00:00:00+00:00
1863-05-18 00:00:00+00:00
1863-05-23 00:00:00+00:00
1863-05-28 00:00:00+00:00
1863-06-02 00:00:00+00:00
1863-06-07 00:00:00+00:00
1863-06-12 00:00:00+00:00
1863-06-17 00:00:00+00:00
1863-06-22 00:00:00+00:00
1863-06-27 00:00:00+00:00
1863-07-02 00:00:00+00:00
1863-07-07 0

...

1936-07-14 00:00:00+00:00
1936-07-19 00:00:00+00:00
1936-07-24 00:00:00+00:00
1936-07-29 00:00:00+00:00
1936-08-03 00:00:00+00:00
1936-08-08 00:00:00+00:00
1936-08-13 00:00:00+00:00
1936-08-18 00:00:00+00:00
1936-08-23 00:00:00+00:00
1936-08-28 00:00:00+00:00
1936-09-02 00:00:00+00:00
1936-09-07 00:00:00+00:00
1936-09-12 00:00:00+00:00
1936-09-17 00:00:00+00:00
1936-09-22 00:00:00+00:00
1936-09-27 00:00:00+00:00
1936-10-02 00:00:00+00:00
1936-10-07 00:00:00+00:00
1936-10-12 00:00:00+00:00
1936-10-17 00:00:00+00:00
1936-10-22 00:00:00+00:00
1936-10-27 00:00:00+00:00
1936-11-01 00:00:00+00:00
1936-11-06 00:00:00+00:00
1936-11-11 00:00:00+00:00
1936-11-16 00:00:00+00:00
1936-11-21 00:00:00+00:00
1936-11-26 00:00:00+00:00
1936-12-01 00:00:00+00:00
1936-12-06 00:00:00+00:00
1936-12-11 00:00:00+00:00
1936-12-16 00:00:00+00:00
1936-12-21 00:00:00+00:00
1936-12-26 00:00:00+00:00
1936-12-31 00:00:00+00:00
1937-01-05 00:00:00+00:00
1937-01-10 00:00:00+00:00
1937-01-15 00:00:00+00:00
1937-01-20 0

1940-11-10 00:00:00+00:00
1940-11-15 00:00:00+00:00
1940-11-20 00:00:00+00:00
1940-11-25 00:00:00+00:00
1940-11-30 00:00:00+00:00
1940-12-05 00:00:00+00:00
1940-12-10 00:00:00+00:00
1940-12-15 00:00:00+00:00
1940-12-20 00:00:00+00:00
1940-12-25 00:00:00+00:00
1940-12-30 00:00:00+00:00
1941-01-04 00:00:00+00:00
1941-01-09 00:00:00+00:00
1941-01-14 00:00:00+00:00
1941-01-19 00:00:00+00:00
1941-01-24 00:00:00+00:00
1941-01-29 00:00:00+00:00
1941-02-03 00:00:00+00:00
1941-02-08 00:00:00+00:00
1941-02-13 00:00:00+00:00
1941-02-18 00:00:00+00:00
1941-02-23 00:00:00+00:00
1941-02-28 00:00:00+00:00
1941-03-05 00:00:00+00:00
1941-03-10 00:00:00+00:00
1941-03-15 00:00:00+00:00
1941-03-20 00:00:00+00:00
1941-03-25 00:00:00+00:00
1941-03-30 00:00:00+00:00
1941-04-04 00:00:00+00:00
1941-04-09 00:00:00+00:00
1941-04-14 00:00:00+00:00
1941-04-19 00:00:00+00:00
1941-04-24 00:00:00+00:00
1941-04-29 00:00:00+00:00
1941-05-04 00:00:00+00:00
1941-05-09 00:00:00+00:00
1941-05-14 00:00:00+00:00
1941-05-19 0

1945-03-09 00:00:00+00:00
1945-03-14 00:00:00+00:00
1945-03-19 00:00:00+00:00
1945-03-24 00:00:00+00:00
1945-03-29 00:00:00+00:00
1945-04-03 00:00:00+00:00
1945-04-08 00:00:00+00:00
1945-04-13 00:00:00+00:00
1945-04-18 00:00:00+00:00
1945-04-23 00:00:00+00:00
1945-04-28 00:00:00+00:00
1945-05-03 00:00:00+00:00
1945-05-08 00:00:00+00:00
1945-05-13 00:00:00+00:00
1945-05-18 00:00:00+00:00
1945-05-23 00:00:00+00:00
1945-05-28 00:00:00+00:00
1945-06-02 00:00:00+00:00
1945-06-07 00:00:00+00:00
1945-06-12 00:00:00+00:00
1945-06-17 00:00:00+00:00
1945-06-22 00:00:00+00:00
1945-06-27 00:00:00+00:00
1945-07-02 00:00:00+00:00
1945-07-07 00:00:00+00:00
1945-07-12 00:00:00+00:00
1945-07-17 00:00:00+00:00
1945-07-22 00:00:00+00:00
1945-07-27 00:00:00+00:00
1945-08-01 00:00:00+00:00
1945-08-06 00:00:00+00:00
1945-08-11 00:00:00+00:00
1945-08-16 00:00:00+00:00
1945-08-21 00:00:00+00:00
1945-08-26 00:00:00+00:00
1945-08-31 00:00:00+00:00
1945-09-05 00:00:00+00:00
1945-09-10 00:00:00+00:00
1945-09-15 0

1949-07-06 00:00:00+00:00
1949-07-11 00:00:00+00:00
1949-07-16 00:00:00+00:00
1949-07-21 00:00:00+00:00
1949-07-26 00:00:00+00:00
1949-07-31 00:00:00+00:00
1949-08-05 00:00:00+00:00
1949-08-10 00:00:00+00:00
1949-08-15 00:00:00+00:00
1949-08-20 00:00:00+00:00
1949-08-25 00:00:00+00:00
1949-08-30 00:00:00+00:00
1949-09-04 00:00:00+00:00
1949-09-09 00:00:00+00:00
1949-09-14 00:00:00+00:00
1949-09-19 00:00:00+00:00
1949-09-24 00:00:00+00:00
1949-09-29 00:00:00+00:00
1949-10-04 00:00:00+00:00
1949-10-09 00:00:00+00:00
1949-10-14 00:00:00+00:00
1949-10-19 00:00:00+00:00
1949-10-24 00:00:00+00:00
1949-10-29 00:00:00+00:00
1949-11-03 00:00:00+00:00
1949-11-08 00:00:00+00:00
1949-11-13 00:00:00+00:00
1949-11-18 00:00:00+00:00
1949-11-23 00:00:00+00:00
1949-11-28 00:00:00+00:00


KeyboardInterrupt: 

In [9]:
data = pd.read_csv('data.csv')
# alternatively, data = celestial_bodies
data.head()

,Unnamed: 0,jupiter_phi,jupiter_r,jupiter_theta,jupiter_x,jupiter_y,jupiter_z,location,mars_phi,mars_r,...,sun_x,sun_y,sun_z,time,venus_phi,venus_r,venus_theta,venus_x,venus_y,venus_z
0,0,3.714044,5.127976,177.157893,-4.912500,0.326557,0.315533,"(4461340.48313723, 919588.07009129, 4449530.22...",26.589000,0.543286,...,0.193749,-0.811200,-0.353117,1850-01-01 00:00:00,-23.797372,1.771032,266.545617,-0.076220,-1.166262,-0.604754
1,1,3.347541,4.874961,175.126237,-4.974439,0.423221,0.347773,"(4461340.48313723, 919588.07009129, 4449530.22...",24.813692,0.642939,...,0.170746,-0.790458,-0.472815,1850-01-06 00:00:00,-24.537521,1.696913,275.553211,0.270615,-1.320716,-0.517961
2,2,4.060894,4.968874,177.682099,-4.858500,0.409376,0.441543,"(4461340.48313723, 919588.07009129, 4449530.22...",27.047331,0.596572,...,0.303053,-0.804421,-0.171615,1850-01-11 00:00:00,-23.539596,1.753273,282.124416,0.134045,-1.357390,-0.677118
3,3,2.179225,4.696147,175.213288,-4.752372,0.232715,0.178511,"(4461340.48313723, 919588.07009129, 4449530.22...",27.705224,0.775200,...,0.567437,-0.727090,-0.392119,1850-01-16 00:00:00,-21.894231,1.623088,288.265631,0.636459,-1.434531,-0.616671
4,4,3.468697,4.539760,176.101327,-4.631363,0.431673,0.274209,"(4461340.48313723, 919588.07009129, 4449530.22...",26.285016,0.689112,...,0.567273,-0.686423,-0.282064,1850-01-21 00:00:00,-22.393396,1.738150,296.151892,0.549981,-1.367793,-0.629948


In [12]:
for name in BODY_NAMES:
    body_data = data[['time', 'location', name + '_phi', name + '_theta', name + '_r', name + '_x', name + '_y', name + '_z']]
    body_data.to_csv(name + '.csv', index=False)

In [ ]:
rows = defaultdict(list)
for name in BODY_NAMES:
    #print(name)
    phi_col = data[name + '_phi']
    theta_col = data[name + '_theta']
    r_col = data[name + '_r']
    for phi, theta, r in zip(phi_col, theta_col, r_col):
        ecliptic = SkyCoord(theta, phi, abs(r), frame='gcrs', unit=('deg', 'deg', 'AU')).transform_to(GeocentricTrueEcliptic())
        coordinates = get_coordinates(ecliptic)
        coord_strings = ['lambda', 'beta', 'delta', 'x', 'y', 'z']
        for i in range(len(coord_strings)):
            c = coordinates[i]
            rows[name + '_' + coord_strings[i]].append(c)

mercury
venus
mars
jupiter
saturn
moon


In [ ]:
final_data = pd.DataFrame(rows)
final_data.to_csv('final_data.csv', index=False)